# **CAP 5768: Introduction to Data Science**
## Final Project Program: Implementing Machine Models

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [10]:
data_cleaned = pd.read_csv('cleaned_dataset.csv')

In [16]:
# Define features and target
X = data_cleaned[['NO2 AQI', 'O3 AQI', 'SO2 AQI', 'CO AQI']]
y = data_cleaned['Overall AQI']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the scalar
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# LinearRegression fit
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Make a prediction
y_pred_lr = lr_model.predict(X_test_scaled)

# Evaluate the model
lr_mse = mean_absolute_error(y_test, y_pred_lr)
lr_r2 = r2_score(y_test, y_pred_lr)

print(f"Mean Squared Error (MSE) of Linear Regression: {lr_mse:.4f}")
print(f"R-Squared (R^2) of Linear Regression: {lr_r2:.4f}")

Mean Squared Error (MSE) of Linear Regression: 5.2664
R-Squared (R^2) of Linear Regression: 0.8709


In [17]:
# Random Forest Regressor
forest_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=4, min_samples_split=20)

# Train model
forest_model.fit(X_train, y_train)

# Make a prediction
y_pred_forest = forest_model.predict(X_test)

# Evaluate
forest_mse = mean_squared_error(y_test, y_pred_forest)
forest_r2 = r2_score(y_test, y_pred_forest)

print(f"Mean Squared Error (MSE) of Random Forest Regressor: {forest_mse:.4f}")
print(f"R^2 Score (R^2) of Random Forest Regressor: {forest_r2:.4f}")

Mean Squared Error (MSE) of Random Forest Regressor: 36.8555
R^2 Score (R^2) of Random Forest Regressor: 0.9089


In [19]:
# Ridge Regression
ridge_model = Ridge(alpha=1.0, fit_intercept=True, solver='auto')

# Train model
ridge_model.fit(X_train, y_train)

# Make a prediction
y_pred_ridge = ridge_model.predict(X_test)

# Evaluate
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_r2 = r2_score(y_test, y_pred_ridge)

print(f"Mean Squared Error (MSE) of Ridge Regression: {forest_mse:.4f}")
print(f"R^2 Score (R^2) of Ridge Regression: {forest_r2:.4f}")

Mean Squared Error (MSE) of Ridge Regression: 36.8555
R^2 Score (R^2) of Ridge Regression: 0.9089


In [ ]:
# Gradicent Boosting Regressor
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Fit model
xgb_model.fit(X_train_scaled, y_train)

# Make a prediction
y_pred_xgb = xgb_model.predict(X_test_scaled)

# Evaluate
xgb_mse = mean_squared_error(y_test, y_pred_xgb)
xgb_r2 = r2_score(y_test, y_pred_xgb)

print(f"Mean Squared Error (MSE) of XGBoost Regressor: {xgb_mse:.4f}")
print(f"R^2 Score (R^2) of XGBoost Regressor: {xgb_r2:.4f}")

Mean Squared Error (MSE) of XGBoost Regressor: 2.5558
R^2 Score (R^2) of XGBoost Regressor: 0.9937


# Model Analysis
* While all models performed within an acceptable range, the XGBoost Regressor model emerged as the top performer. This is evident when analyzing both its R-squared score and its Mean Squared Error (MSE) compared to the other models. The results indicate that it accurately predicted 99.37% of the variance in the data, as shown by the R-squared score of 0.9937. Furthermore, it exhibited a relatively small error margin, with a Mean Squared Error of 2.558, which is notably good for our dataset.

In [ ]:
# Define the parameters grid for XGBoost
param_grid_xgb = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Perform Grid Search with 5-fold cross-validation
grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=5, scoring='neg_mean_squared_error')

# Fit the grid search
grid_search_xgb.fit(X_train_scaled, y_train.values.ravel())

# Print the best parameters and best score
print("Best parameters for XGBoost Regressor:", grid_search_xgb.best_params_)
print(f"Best score for XGBoost Regressor: {grid_search_xgb.best_score_:.4f}")

Best parameters for XGBoost Regressor: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}
Best score for XGBoost Regressor: -2.2736


In [24]:
best_xgb_model = grid_search_xgb.best_estimator_

y_pred_best_xgb = best_xgb_model.predict(X_test_scaled)

best_xgb_mse = mean_squared_error(y_test, y_pred_best_xgb)
best_xgb_r2 = r2_score(y_test, y_pred_best_xgb)

print(f"Mean Squared Error (MSE) of Best XGBoost Regressor: {best_xgb_mse:.4f}")
print(f"R^2 Score (R^2) of Best XGBoost Regressor: {best_xgb_r2:.4f}")

Mean Squared Error (MSE) of Best XGBoost Regressor: 2.3270
R^2 Score (R^2) of Best XGBoost Regressor: 0.9942


# Hyperparameter Analysis
* To identify the optimal parameters for our top-performing model, XGBoost, we conducted a Grid Search utilizing 5-fold cross-validation. The parameters identified through this process allowed for a slight improvement in the model's performance.
* Ideally, we would have liked to further expand the Grid Search by incorporating additional parameters such as reg_alpha and reg_lambda, among others. However, time constraints limited our ability to perform this more extensive search